In [73]:
import requests
import requests_cache

import pandas as pd
import networkx as nx

In [74]:
from fake_useragent import UserAgent
from redis import StrictRedis

from functools import partial, wraps
from operator import attrgetter

In [3]:
requests_cache.install_cache(
    'nba-rank',
    expire_after=24*60*60,
    backend='sqlite'
)

In [4]:
def apply_to_output(callback, *cb_args, **cb_kwargs):

    def decorator(func):

        @wraps(func)
        def new_func(*args, **kwargs):
            return callback(func(*args, **kwargs), *cb_args, **cb_kwargs)

        return new_func

    return decorator

In [5]:
def get_json(url, *args, **kwargs):
    r = requests.get(url=url, *args, **kwargs)
    r.raise_for_status()
    return r.json()

In [6]:
ua = UserAgent()

In [7]:
get_json = partial(get_json, headers={'User-Agent': ua.google})

In [8]:
def nested_dict_from_lst(lst, key):
    return {d.pop(key): d for d in lst}

In [75]:
def normalize_dict(dct, frame_name_key='name', frames_key='resultSets',
                   frames_default=[]):
    return nested_dict_from_lst(dct.get(frames_key, frames_default),
                                key=frame_name_key)

In [76]:
def to_data_frame(frames_dict, frame_name, frame_index=None,
                  frame_data_key='rowSet', frame_columns_key='headers'):
    return pd.DataFrame.from_records(
               data=frames_dict[frame_name][frame_data_key],
               columns=frames_dict[frame_name][frame_columns_key],
               index=frame_index).dropna()

In [77]:
@apply_to_output(to_data_frame,
                 frame_name='LeagueGameLog', frame_index='GAME_ID')
@apply_to_output(normalize_dict)
def games_df(league_id='00', season='2015-16', season_type='Regular Season',
             team=True, sort_by='date', ascending=True):

    return get_json(url='http://stats.nba.com/stats/LeagueGameLog',
                    params={'LeagueID': league_id,
                            'PlayerOrTeam': 'T' if team else 'P',
                            'Season': season,
                            'SeasonType': season_type,
                            'Sorter': sort_by.upper(),
                            'Direction': 'ASC' if ascending else 'DESC'})

In [78]:
def pivot_games_df(df):

    primary_columns = [
        'TEAM_ID',
        'FGM',
        'FGA',
        'FG3M',
        'FG3A',
        'FTM',
        'FTA',
        'OREB',
        'DREB',
        'AST',
        'STL',
        'BLK',
        'TOV',
        'PF',
        'PTS',
    ]

    visitors = df[df['MATCHUP'].str.contains('@')]['TEAM_ID']

    df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'], format='%Y-%m-%d')
    df = df.reset_index('GAME_ID')

    pivot_df = df.pivot_table(index=['GAME_ID', 'SEASON_ID', 'GAME_DATE', 'MIN'],
                              columns='WL',
                              values=primary_columns,
                              aggfunc=lambda s: s.iloc[0])
    pivot_df = pivot_df.reset_index(['SEASON_ID', 'GAME_DATE', 'MIN'])
    pivot_df['VISITOR'] = visitors

    return pivot_df

In [79]:
@apply_to_output(to_data_frame, frame_name='TeamYears', frame_index='TEAM_ID')
@apply_to_output(normalize_dict)
def teams_df(league_id='00'):

    # Make Request
    return get_json(url='http://stats.nba.com/stats/commonTeamYears',
                    params={'LeagueID': league_id})

In [80]:
@apply_to_output(to_data_frame,
                 frame_name='TeamInfoCommon', frame_index='TEAM_ID')
@apply_to_output(normalize_dict)
def team_details_df(team_id, league_id='00', season='2015-16',
                    season_type='Regular Season'):

    return get_json(url='http://stats.nba.com/stats/TeamInfoCommon',
                    params={'LeagueID': league_id,
                            'TeamID': team_id,
                            'Season': season,
                            'SeasonType': season_type})

In [81]:
def teams_details_df(league_id='00', season='2015-16',
                     season_type='Regular Season'):

    return pd.concat(map(partial(team_details_df,
                                 league_id=league_id,
                                 season=season,
                                 season_type=season_type),
                         teams(league_id=league_id).index))

In [83]:
G = nx.MultiDiGraph()

In [84]:
G.add_nodes_from([(1, dict(foo=42)), (5, dict(bar=21))])

In [85]:
list(G.nodes(data=True))

[(1, {'foo': 42}), (5, {'bar': 21})]

In [86]:
G.node[1]

{'foo': 42}

In [87]:
teams_details_df().to_dict(orient='index')

{1610612737: {'CONF_RANK': 4,
  'DIV_RANK': 2,
  'L': 34,
  'MAX_YEAR': '2016',
  'MIN_YEAR': '1949',
  'PCT': 0.585,
  'SEASON_YEAR': '2015-16',
  'TEAM_ABBREVIATION': 'ATL',
  'TEAM_CITY': 'Atlanta',
  'TEAM_CODE': 'hawks',
  'TEAM_CONFERENCE': 'East',
  'TEAM_DIVISION': 'Southeast',
  'TEAM_NAME': 'Hawks',
  'W': 48},
 1610612738: {'CONF_RANK': 5,
  'DIV_RANK': 2,
  'L': 34,
  'MAX_YEAR': '2016',
  'MIN_YEAR': '1946',
  'PCT': 0.585,
  'SEASON_YEAR': '2015-16',
  'TEAM_ABBREVIATION': 'BOS',
  'TEAM_CITY': 'Boston',
  'TEAM_CODE': 'celtics',
  'TEAM_CONFERENCE': 'East',
  'TEAM_DIVISION': 'Atlantic',
  'TEAM_NAME': 'Celtics',
  'W': 48},
 1610612739: {'CONF_RANK': 1,
  'DIV_RANK': 1,
  'L': 25,
  'MAX_YEAR': '2016',
  'MIN_YEAR': '1970',
  'PCT': 0.695,
  'SEASON_YEAR': '2015-16',
  'TEAM_ABBREVIATION': 'CLE',
  'TEAM_CITY': 'Cleveland',
  'TEAM_CODE': 'cavaliers',
  'TEAM_CONFERENCE': 'East',
  'TEAM_DIVISION': 'Central',
  'TEAM_NAME': 'Cavaliers',
  'W': 57},
 1610612740: {'CONF_R

In [138]:
G = nx.MultiDiGraph()

In [139]:
G.add_nodes_from(teams_details_df().to_dict(orient='index').items())

In [140]:
list(G.nodes(data=True))

[(1610612737,
  {'CONF_RANK': 4,
   'DIV_RANK': 2,
   'L': 34,
   'MAX_YEAR': '2016',
   'MIN_YEAR': '1949',
   'PCT': 0.585,
   'SEASON_YEAR': '2015-16',
   'TEAM_ABBREVIATION': 'ATL',
   'TEAM_CITY': 'Atlanta',
   'TEAM_CODE': 'hawks',
   'TEAM_CONFERENCE': 'East',
   'TEAM_DIVISION': 'Southeast',
   'TEAM_NAME': 'Hawks',
   'W': 48}),
 (1610612738,
  {'CONF_RANK': 5,
   'DIV_RANK': 2,
   'L': 34,
   'MAX_YEAR': '2016',
   'MIN_YEAR': '1946',
   'PCT': 0.585,
   'SEASON_YEAR': '2015-16',
   'TEAM_ABBREVIATION': 'BOS',
   'TEAM_CITY': 'Boston',
   'TEAM_CODE': 'celtics',
   'TEAM_CONFERENCE': 'East',
   'TEAM_DIVISION': 'Atlantic',
   'TEAM_NAME': 'Celtics',
   'W': 48}),
 (1610612739,
  {'CONF_RANK': 1,
   'DIV_RANK': 1,
   'L': 25,
   'MAX_YEAR': '2016',
   'MIN_YEAR': '1970',
   'PCT': 0.695,
   'SEASON_YEAR': '2015-16',
   'TEAM_ABBREVIATION': 'CLE',
   'TEAM_CITY': 'Cleveland',
   'TEAM_CODE': 'cavaliers',
   'TEAM_CONFERENCE': 'East',
   'TEAM_DIVISION': 'Central',
   'TEAM_NAME

In [107]:
G.node[1610612738]['TEAM_NAME']

'Celtics'

In [108]:
games_pivot(games())

SEASON_ID  GAME_DATE  MIN     TEAM_ID             FGM      FGA  \
WL                                             L           W   L   W    L   
GAME_ID                                                                     
0021500001     22015 2015-10-27  240  1610612737  1610612765  37  37   82   
0021500002     22015 2015-10-27  240  1610612739  1610612741  38  37   94   
0021500003     22015 2015-10-27  240  1610612740  1610612744  35  41   83   
0021500004     22015 2015-10-28  240  1610612753  1610612764  37  33  100   
0021500005     22015 2015-10-28  240  1610612755  1610612738  34  39   83   
0021500006     22015 2015-10-28  240  1610612751  1610612741  39  42   93   
0021500007     22015 2015-10-28  240  1610612762  1610612765  35  30   75   
0021500008     22015 2015-10-28  240  1610612766  1610612748  33  36   84   
0021500009     22015 2015-10-28  240  1610612754  1610612761  32  36   86   
0021500010     22015 2015-10-28  240  1610612745  1610612743  30  40   87   
0021500011     22015 2015-10-28  240  1610612763  1610612739  29  41   82   
0021500012     22015 2015-10-28  240  1610612749  1610612752  30  42   86   
0021500013     22015 2015-10-28  240  1610612759  1610612760  45  42   93   
0021500014     22015 2015-10-28  240  1610612756  1610612742  34  40   87   
0021500015     22015 2015-10-28  240  1610612740  1610612757  36  44   91   
0021500016     22015 2015-10-28  240  1610612758  1610612746  41  42   94   
0021500017     22015 2015-10-28  240  1610612747  1610612750  35  38   90   
0021500018     22015 2015-10-29  240  1610612754  1610612763  37  41   78   
0021500019     22015 2015-10-29  240  1610612752  1610612737  38  42   93   
0021500020     22015 2015-10-29  240  1610612742  1610612746  35  38   97   
0021500021     22015 2015-10-30  240  1610612748  1610612739  33  39   77   
0021500022     22015 2015-10-30  290  1610612753  1610612760  47  50  106   
0021500023     22015 2015-10-30  240  1610612755  1610612762  19  36   63   
0021500024     22015 2015-10-30  240  1610612738  1610612761  32  36   85   
0021500025     22015 2015-10-30  265  1610612741  1610612765  34  39   84   
0021500026     22015 2015-10-30  240  1610612766  1610612737  36  36   88   
0021500027     22015 2015-10-30  240  1610612749  1610612764  42  35   82   
0021500028     22015 2015-10-30  240  1610612751  1610612759  31  40   77   
0021500029     22015 2015-10-30  240  1610612743  1610612750  29  30   95   
0021500030     22015 2015-10-30  240  1610612745  1610612744  30  43   82   
...              ...        ...  ...         ...         ...  ..  ..  ...   
0021501201     22015 2016-04-10  240  1610612759  1610612744  34  34   90   
0021501202     22015 2016-04-10  240  1610612752  1610612761  32  37   89   
0021501203     22015 2016-04-11  240  1610612737  1610612739  39  40   87   
0021501204     22015 2016-04-11  240  1610612749  1610612753  41  41   78   
0021501205     22015 2016-04-11  240  1610612738  1610612766  34  41   89   
0021501206     22015 2016-04-11  240  1610612751  1610612764  44  46   98   
0021501207     22015 2016-04-11  240  1610612750  1610612745  38  48   83   
0021501208     22015 2016-04-11  240  1610612740  1610612741  41  44   80   
0021501209     22015 2016-04-11  240  1610612747  1610612760  23  42   81   
0021501210     22015 2016-04-11  240  1610612762  1610612742  35  36   77   
0021501211     22015 2016-04-11  240  1610612756  1610612758  41  41  100   
0021501212     22015 2016-04-12  240  1610612752  1610612754  36  41   80   
0021501213     22015 2016-04-12  240  1610612765  1610612748  34  38   84   
0021501214     22015 2016-04-12  240  1610612755  1610612761  35  48   77   
0021501215     22015 2016-04-12  265  1610612760  1610612759  35  37   87   
0021501216     22015 2016-04-12  240  1610612763  1610612746  35  41   86   
0021501217     22015 2016-04-13  240  1610612748  1610612738  38  37   88   
0021501218     22015 2016-04-13  240  1610612751  1610612761  39  41   89   
0021501

In [114]:
nx.readwrite.node_link_data(G).get('nodes')

[{'name': 1610612737},
 {'name': 1610612738},
 {'name': 1610612739},
 {'name': 1610612740},
 {'name': 1610612741},
 {'name': 1610612742},
 {'name': 1610612743},
 {'name': 1610612744},
 {'name': 1610612745},
 {'name': 1610612746},
 {'name': 1610612747},
 {'name': 1610612748},
 {'name': 1610612749},
 {'name': 1610612750},
 {'name': 1610612751},
 {'name': 1610612752},
 {'name': 1610612753},
 {'name': 1610612754},
 {'name': 1610612755},
 {'name': 1610612756},
 {'name': 1610612757},
 {'name': 1610612758},
 {'name': 1610612759},
 {'name': 1610612760},
 {'name': 1610612761},
 {'name': 1610612762},
 {'name': 1610612763},
 {'name': 1610612764},
 {'name': 1610612765},
 {'name': 1610612766}]

In [141]:
H = nx.from_pandas_dataframe(df=games_pivot(games()),
                             source=('TEAM_ID', 'L'),
                             target=('TEAM_ID', 'W'),
                             edge_attr=True,
                             create_using=nx.MultiDiGraph())

In [142]:
list(H.nodes())[:5]

[1610612737, 1610612738, 1610612739, 1610612740, 1610612741]

In [143]:
final = nx.compose(G=G, H=H)

In [144]:
nx.readwrite.node_link_data(final).get('nodes')

[{'CONF_RANK': 4,
  'DIV_RANK': 2,
  'L': 34,
  'MAX_YEAR': '2016',
  'MIN_YEAR': '1949',
  'PCT': 0.585,
  'SEASON_YEAR': '2015-16',
  'TEAM_ABBREVIATION': 'ATL',
  'TEAM_CITY': 'Atlanta',
  'TEAM_CODE': 'hawks',
  'TEAM_CONFERENCE': 'East',
  'TEAM_DIVISION': 'Southeast',
  'TEAM_NAME': 'Hawks',
  'W': 48,
  'name': 1610612737},
 {'CONF_RANK': 5,
  'DIV_RANK': 2,
  'L': 34,
  'MAX_YEAR': '2016',
  'MIN_YEAR': '1946',
  'PCT': 0.585,
  'SEASON_YEAR': '2015-16',
  'TEAM_ABBREVIATION': 'BOS',
  'TEAM_CITY': 'Boston',
  'TEAM_CODE': 'celtics',
  'TEAM_CONFERENCE': 'East',
  'TEAM_DIVISION': 'Atlantic',
  'TEAM_NAME': 'Celtics',
  'W': 48,
  'name': 1610612738},
 {'CONF_RANK': 1,
  'DIV_RANK': 1,
  'L': 25,
  'MAX_YEAR': '2016',
  'MIN_YEAR': '1970',
  'PCT': 0.695,
  'SEASON_YEAR': '2015-16',
  'TEAM_ABBREVIATION': 'CLE',
  'TEAM_CITY': 'Cleveland',
  'TEAM_CODE': 'cavaliers',
  'TEAM_CONFERENCE': 'East',
  'TEAM_DIVISION': 'Central',
  'TEAM_NAME': 'Cavaliers',
  'W': 57,
  'name': 1610

In [49]:
nx.readwrite.node_link_data(final).keys()

dict_keys(['nodes', 'links', 'directed', 'multigraph', 'graph'])

In [65]:
nx.readwrite.node_link_data(final).get('links')

[{('OREB', 'L'): 5,
  ('MIN', ''): 240,
  ('STL', 'L'): 3,
  ('FGA', 'L'): 76,
  ('FGM', 'W'): 42,
  ('TEAM_ID', 'L'): 1610612737,
  ('FG3A', 'W'): 33,
  ('FTA', 'L'): 20,
  'target': 1,
  ('PTS', 'W'): 106,
  ('SEASON_ID', ''): '22015',
  ('TOV', 'L'): 17,
  ('DREB', 'L'): 30,
  ('BLK', 'W'): 7,
  ('FG3A', 'L'): 20,
  ('PF', 'W'): 18,
  ('DREB', 'W'): 33,
  ('FTM', 'W'): 11,
  ('TEAM_ID', 'W'): 1610612738,
  ('PF', 'L'): 14,
  'key': 0,
  ('AST', 'L'): 26,
  ('BLK', 'L'): 2,
  ('GAME_DATE', ''): Timestamp('2015-11-13 00:00:00'),
  ('VISITOR', ''): 1610612737,
  ('FTM', 'L'): 15,
  ('STL', 'W'): 10,
  ('FG3M', 'W'): 11,
  ('OREB', 'W'): 17,
  ('AST', 'W'): 27,
  ('FGA', 'W'): 103,
  'source': 0,
  ('FTA', 'W'): 12,
  ('PTS', 'L'): 93,
  ('FG3M', 'L'): 6,
  ('TOV', 'W'): 10,
  ('FGM', 'L'): 36},
 {('OREB', 'L'): 12,
  ('MIN', ''): 240,
  ('STL', 'L'): 10,
  ('FGA', 'L'): 86,
  ('FGM', 'W'): 41,
  ('TEAM_ID', 'L'): 1610612737,
  ('FG3A', 'W'): 29,
  ('FTA', 'L'): 25,
  'target': 2,
  ('P

In [66]:
final.node[1610612738]

{'CONF_RANK': 5,
 'DIV_RANK': 2,
 'L': 34,
 'MAX_YEAR': '2016',
 'MIN_YEAR': '1946',
 'PCT': 0.585,
 'SEASON_YEAR': '2015-16',
 'TEAM_ABBREVIATION': 'BOS',
 'TEAM_CITY': 'Boston',
 'TEAM_CODE': 'celtics',
 'TEAM_CONFERENCE': 'East',
 'TEAM_DIVISION': 'Atlantic',
 'TEAM_NAME': 'Celtics',
 'W': 48}

In [67]:
final.node.keys()

dict_keys([1610612737, 1610612738, 1610612739, 1610612740, 1610612741, 1610612742, 1610612743, 1610612744, 1610612745, 1610612746, 1610612747, 1610612748, 1610612749, 1610612750, 1610612751, 1610612752, 1610612753, 1610612754, 1610612755, 1610612756, 1610612757, 1610612758, 1610612759, 1610612760, 1610612761, 1610612762, 1610612763, 1610612764, 1610612765, 1610612766])

In [128]:
nx.readwrite.json_graph.node_link_data(final)

{'directed': True,
 'graph': {'name': 'compose( ,  )'},
 'links': [{('OREB', 'L'): 5,
   ('MIN', ''): 240,
   ('STL', 'L'): 3,
   ('FGA', 'L'): 76,
   ('FGM', 'W'): 42,
   ('TEAM_ID', 'L'): 1610612737,
   ('FG3A', 'W'): 33,
   ('FTA', 'L'): 20,
   'target': 1,
   ('PTS', 'W'): 106,
   ('SEASON_ID', ''): '22015',
   ('TOV', 'L'): 17,
   ('DREB', 'L'): 30,
   ('BLK', 'W'): 7,
   ('FG3A', 'L'): 20,
   ('PF', 'W'): 18,
   ('DREB', 'W'): 33,
   ('FTM', 'W'): 11,
   ('TEAM_ID', 'W'): 1610612738,
   ('PF', 'L'): 14,
   'key': 0,
   ('AST', 'L'): 26,
   ('BLK', 'L'): 2,
   ('GAME_DATE', ''): Timestamp('2015-11-13 00:00:00'),
   ('VISITOR', ''): 1610612737,
   ('FTM', 'L'): 15,
   ('STL', 'W'): 10,
   ('FG3M', 'W'): 11,
   ('OREB', 'W'): 17,
   ('AST', 'W'): 27,
   ('FGA', 'W'): 103,
   'source': 0,
   ('FTA', 'W'): 12,
   ('PTS', 'L'): 93,
   ('FG3M', 'L'): 6,
   ('TOV', 'W'): 10,
   ('FGM', 'L'): 36},
  {('OREB', 'L'): 12,
   ('MIN', ''): 240,
   ('STL', 'L'): 10,
   ('FGA', 'L'): 86,
   ('FG

In [130]:
import json

In [131]:
json.dumps(nx.readwrite.json_graph.node_link_data(final))

TypeError: keys must be a string

In [146]:
nx.readwrite.json_graph.node_link_data(final)

{'directed': True,
 'graph': {'name': 'compose( ,  )'},
 'links': [{('OREB', 'L'): 5,
   ('MIN', ''): 240,
   ('STL', 'L'): 3,
   ('FGA', 'L'): 76,
   ('FGM', 'W'): 42,
   ('TEAM_ID', 'L'): 1610612737,
   ('FG3A', 'W'): 33,
   ('FTA', 'L'): 20,
   'target': 1,
   ('PTS', 'W'): 106,
   ('SEASON_ID', ''): '22015',
   ('TOV', 'L'): 17,
   ('DREB', 'L'): 30,
   ('BLK', 'W'): 7,
   ('FG3A', 'L'): 20,
   ('PF', 'W'): 18,
   ('DREB', 'W'): 33,
   ('FTM', 'W'): 11,
   ('TEAM_ID', 'W'): 1610612738,
   ('PF', 'L'): 14,
   'key': 0,
   ('AST', 'L'): 26,
   ('BLK', 'L'): 2,
   ('GAME_DATE', ''): Timestamp('2015-11-13 00:00:00'),
   ('VISITOR', ''): 1610612737,
   ('FTM', 'L'): 15,
   ('STL', 'W'): 10,
   ('FG3M', 'W'): 11,
   ('OREB', 'W'): 17,
   ('AST', 'W'): 27,
   ('FGA', 'W'): 103,
   'source': 0,
   ('FTA', 'W'): 12,
   ('PTS', 'L'): 93,
   ('FG3M', 'L'): 6,
   ('TOV', 'W'): 10,
   ('FGM', 'L'): 36},
  {('OREB', 'L'): 12,
   ('MIN', ''): 240,
   ('STL', 'L'): 10,
   ('FGA', 'L'): 86,
   ('FG

In [153]:
import yaml
import requests
import requests_cache
import pandas as pd
import networkx as nx

from fake_useragent import UserAgent
from redis import StrictRedis

from functools import partial, wraps
from operator import attrgetter


requests_cache.install_cache(
    'nba-rank',
    expire_after=24*60*60,
    backend='sqlite'
)


def apply_to_output(callback, *cb_args, **cb_kwargs):

    def decorator(func):

        @wraps(func)
        def new_func(*args, **kwargs):
            return callback(func(*args, **kwargs), *cb_args, **cb_kwargs)

        return new_func

    return decorator


def get_json(url, *args, **kwargs):
    r = requests.get(url=url, *args, **kwargs)
    r.raise_for_status()
    return r.json()


def nested_dict_from_lst(lst, key):
    return {d.pop(key): d for d in lst}


def normalize_dict(dct, frame_name_key='name', frames_key='resultSets',
                   frames_default=[]):
    return nested_dict_from_lst(dct.get(frames_key, frames_default),
                                key=frame_name_key)


def to_data_frame(frames_dict, frame_name, frame_index=None,
                  frame_data_key='rowSet', frame_columns_key='headers'):
    return pd.DataFrame.from_records(
               data=frames_dict[frame_name][frame_data_key],
               columns=frames_dict[frame_name][frame_columns_key],
               index=frame_index).dropna()


ua = UserAgent()
get_json = partial(get_json, headers={'User-Agent': ua.google})


@apply_to_output(to_data_frame,
                 frame_name='LeagueGameLog', frame_index='GAME_ID')
@apply_to_output(normalize_dict)
def games_df(league_id='00', season='2015-16', season_type='Regular Season',
             team=True, sort_by='date', ascending=True):

    return get_json(url='http://stats.nba.com/stats/LeagueGameLog',
                    params={'LeagueID': league_id,
                            'PlayerOrTeam': 'T' if team else 'P',
                            'Season': season,
                            'SeasonType': season_type,
                            'Sorter': sort_by.upper(),
                            'Direction': 'ASC' if ascending else 'DESC'})


def pivot_games_df(df):

    # primary_columns = [
    #     'TEAM_ID',
    #     'FGM',
    #     'FGA',
    #     'FG3M',
    #     'FG3A',
    #     'FTM',
    #     'FTA',
    #     'OREB',
    #     'DREB',
    #     'AST',
    #     'STL',
    #     'BLK',
    #     'TOV',
    #     'PF',
    #     'PTS',
    # ]
    primary_columns = 'TEAM_ID'

    visitors = df[df['MATCHUP'].str.contains('@')]['TEAM_ID']

    df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'], format='%Y-%m-%d')

    df = df.reset_index('GAME_ID')

    pivot_df = df.pivot_table(index=['GAME_ID', 'SEASON_ID', 'GAME_DATE', 'MIN'],
                              columns='WL',
                              values=primary_columns,
                              aggfunc=lambda s: s.iloc[0])
    pivot_df = pivot_df.reset_index(['SEASON_ID', 'GAME_DATE', 'MIN'])
    pivot_df['VISITOR'] = visitors

    return pivot_df


@apply_to_output(to_data_frame, frame_name='TeamYears', frame_index='TEAM_ID')
@apply_to_output(normalize_dict)
def teams_df(league_id='00'):

    # Make Request
    return get_json(url='http://stats.nba.com/stats/commonTeamYears',
                    params={'LeagueID': league_id})


@apply_to_output(to_data_frame,
                 frame_name='TeamInfoCommon', frame_index='TEAM_ID')
@apply_to_output(normalize_dict)
def team_details_df(team_id, league_id='00', season='2015-16',
                    season_type='Regular Season'):

    return get_json(url='http://stats.nba.com/stats/TeamInfoCommon',
                    params={'LeagueID': league_id,
                            'TeamID': team_id,
                            'Season': season,
                            'SeasonType': season_type})


def teams_details_df(league_id='00', season='2015-16',
                     season_type='Regular Season'):

    return pd.concat(map(partial(team_details_df,
                                 league_id=league_id,
                                 season=season,
                                 season_type=season_type),
                         teams_df(league_id=league_id).index))


def g(league_id='00', season='2015-16', season_type='Regular Season',
      create_using=None):

    if create_using is None:
        create_using = nx.MultiDiGraph()

    df = teams_details_df(league_id=league_id, season=season,
                          season_type=season_type)

    G = nx.convert._prep_create_using(create_using)
    G.add_nodes_from(df.to_dict(orient='index').items())

    H = nx.from_pandas_dataframe(df=pivot_games_df(
                                      games_df(league_id=league_id,
                                               season=season,
                                               season_type=season_type)),
                                 # source=('TEAM_ID', 'L'),
                                 # target=('TEAM_ID', 'W'),
                                 source='L',
                                 target='W',
                                 edge_attr=True,
                                 create_using=create_using)
#                                  create_using=nx.MultiDiGraph())

    return nx.compose(G=G, H=H, name='win_loss_network')


In [154]:
json.dumps(nx.readwrite.json_graph.node_link_data(g()))

TypeError: Timestamp('2015-11-13 00:00:00') is not JSON serializable

In [156]:
teams_details_df().index

Int64Index([1610612737, 1610612738, 1610612739, 1610612740, 1610612741,
            1610612742, 1610612743, 1610612744, 1610612745, 1610612747,
            1610612748, 1610612749, 1610612750, 1610612751, 1610612752,
            1610612753, 1610612754, 1610612755, 1610612756, 1610612757,
            1610612759, 1610612760, 1610612761, 1610612762, 1610612763,
            1610612764, 1610612765, 1610612766, 1610612746, 1610612758],
           dtype='int64', name='TEAM_ID')

In [158]:
@apply_to_output(normalize_dict)
def teams(league_id='00'):

    # Make Request
    return get_json(url='http://stats.nba.com/stats/commonTeamYears',
                    params={'LeagueID': league_id})


In [159]:
teams()

{'TeamYears': {'headers': ['LEAGUE_ID',
   'TEAM_ID',
   'MIN_YEAR',
   'MAX_YEAR',
   'ABBREVIATION'],
  'rowSet': [['00', 1610612737, '1949', '2016', 'ATL'],
   ['00', 1610612738, '1946', '2016', 'BOS'],
   ['00', 1610612739, '1970', '2016', 'CLE'],
   ['00', 1610612740, '2002', '2016', 'NOP'],
   ['00', 1610612741, '1966', '2016', 'CHI'],
   ['00', 1610612742, '1980', '2016', 'DAL'],
   ['00', 1610612743, '1976', '2016', 'DEN'],
   ['00', 1610612744, '1946', '2016', 'GSW'],
   ['00', 1610612745, '1967', '2016', 'HOU'],
   ['00', 1610612747, '1948', '2016', 'LAL'],
   ['00', 1610612748, '1988', '2016', 'MIA'],
   ['00', 1610612749, '1968', '2016', 'MIL'],
   ['00', 1610612750, '1989', '2016', 'MIN'],
   ['00', 1610612751, '1976', '2016', 'BKN'],
   ['00', 1610612752, '1946', '2016', 'NYK'],
   ['00', 1610612753, '1989', '2016', 'ORL'],
   ['00', 1610612754, '1976', '2016', 'IND'],
   ['00', 1610612755, '1949', '2016', 'PHI'],
   ['00', 1610612756, '1968', '2016', 'PHX'],
   ['00', 16

In [180]:
df = pd.DataFrame(data=teams()['TeamYears']['rowSet'], 
                  columns=teams()['TeamYears']['headers'],
                  dtype=str)

In [181]:
df['TEAM_ID'].dtype

dtype('O')

In [183]:
df.set_index('TEAM_ID').index.astype(str)

Index(['1610612737', '1610612738', '1610612739', '1610612740', '1610612741',
       '1610612742', '1610612743', '1610612744', '1610612745', '1610612747',
       '1610612748', '1610612749', '1610612750', '1610612751', '1610612752',
       '1610612753', '1610612754', '1610612755', '1610612756', '1610612757',
       '1610612759', '1610612760', '1610612761', '1610612762', '1610612763',
       '1610612764', '1610612765', '1610612766', '1610612746', '1610612758',
       '1610610024', '1610610023', '1610610029', '1610610025', '1610610031',
       '1610610034', '1610610036', '1610610027', '1610610030', '1610610033',
       '1610610037', '1610610026', '1610610028', '1610610032', '1610610035'],
      dtype='object', name='TEAM_ID')

In [174]:
df.index

Int64Index([1610612737, 1610612738, 1610612739, 1610612740, 1610612741,
            1610612742, 1610612743, 1610612744, 1610612745, 1610612747,
            1610612748, 1610612749, 1610612750, 1610612751, 1610612752,
            1610612753, 1610612754, 1610612755, 1610612756, 1610612757,
            1610612759, 1610612760, 1610612761, 1610612762, 1610612763,
            1610612764, 1610612765, 1610612766, 1610612746, 1610612758,
            1610610024, 1610610023, 1610610029, 1610610025, 1610610031,
            1610610034, 1610610036, 1610610027, 1610610030, 1610610033,
            1610610037, 1610610026, 1610610028, 1610610032, 1610610035],
           dtype='int64', name='TEAM_ID')